# Kernel（核函数）详解## Comprehensive Guide to Kernel Functions**作者 (Author)**: YueYueYYY  **日期 (Date)**: 2025年12月  **参考 (Reference)**: Klabunde et al. (2025) - Similarity of Neural Network Models---## 📚 目录 (Table of Contents)1. [什么是Kernel?](#section1)2. [为什么需要Kernel?](#section2)3. [常见Kernel类型](#section3)4. [Kernel在CKA中的应用](#section4)5. [代码实现](#section5)6. [可视化理解](#section6)7. [实际应用案例](#section7)8. [总结](#section8)

---<a id='section1'></a>## 1. 什么是Kernel? (What is a Kernel?)### 最简单的理解 (Simple Understanding)**Kernel（核函数）= 相似度计算器**它回答一个问题：**两个数据点有多相似？**### 数学定义 (Mathematical Definition)$$K: \mathbb{R}^D \times \mathbb{R}^D \rightarrow \mathbb{R}$$- **输入 (Input)**: 两个向量 $\mathbf{x}, \mathbf{x}' \in \mathbb{R}^D$- **输出 (Output)**: 一个相似度分数 $K(\mathbf{x}, \mathbf{x}')$### 核心性质 (Key Properties)1. **对称性 (Symmetry)**: $K(\mathbf{x}, \mathbf{x}') = K(\mathbf{x}', \mathbf{x})$2. **正定性 (Positive Definiteness)**: 核矩阵是半正定的3. **非负性 (Non-negativity)**: 通常 $K(\mathbf{x}, \mathbf{x}) \geq 0$### 直观例子 (Intuitive Example)```学生A: [数学:90, 物理:85, 化学:88]学生B: [数学:92, 物理:83, 化学:90]K(A, B) = 90×92 + 85×83 + 88×90 = 23,195分数越高 → 越相似```

---<a id='section2'></a>## 2. 为什么需要Kernel? (Why Do We Need Kernels?)### 核心思想: "Kernel Trick" (核技巧)**问题场景 (Problem Scenario)**:```原始空间中数据不可分 (Data not separable in original space)      ↓映射到高维空间 (Map to higher dimension)      ↓在高维空间中可分 (Separable in high dimension)```### Kernel的魔力 (The Magic of Kernels)**关键 Insight**:$$K(\mathbf{x}, \mathbf{x}') = \langle \phi(\mathbf{x}), \phi(\mathbf{x}') \rangle$$我们**不需要**知道 $\phi$ 是什么！只需要计算 $K$ 就够了！**计算复杂度对比 (Computational Complexity)**:- ❌ 显式映射: $O(D_{high})$ - 高维可能是无穷维！- ✅ 使用Kernel: $O(D_{original})$ - 只在原始空间计算！### 实际例子 (Practical Example)想象你有**基因表达数据**：```python# 两个实验条件的基因表达condition_A = [gene1_expr, gene2_expr, ..., gene1000_expr]  # 1000个基因condition_B = [gene1_expr, gene2_expr, ..., gene1000_expr]# 用Kernel计算相似度similarity = K(condition_A, condition_B)# → 告诉你两个条件的基因表达模式有多相似```

---<a id='section3'></a>## 3. 常见Kernel类型 (Common Kernel Types)### 3.1 Linear Kernel (线性核) ⭐ **论文默认**$$K(\mathbf{x}, \mathbf{x}') = \mathbf{x}^T \mathbf{x}'$$**特点 (Features)**:- 就是普通的点积/内积- 不做任何变换- 在CKA中最常用- 计算效率最高**应用场景 (Use Cases)**:- 数据本身已经线性可分- 想要保持原始空间的几何关系- 论文中的默认选择---### 3.2 RBF Kernel (径向基函数核/高斯核)$$K(\mathbf{x}, \mathbf{x}') = \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}'\|^2}{2\sigma^2}\right)$$**参数 (Parameters)**:- $\sigma$ (sigma): 带宽参数，控制"相似"的范围**特点 (Features)**:- 输出范围: [0, 1]- $K(\mathbf{x}, \mathbf{x}) = 1$ (自己和自己最相似)- 距离越远，相似度越低- 映射到**无穷维**空间！**$\sigma$ 的影响**:```σ = 0.1 (小): 只有很近的点才相似 (局部)    x  →  [1.0, 0.1, 0.0, 0.0]    σ = 10 (大): 远的点也相似 (全局)    x  →  [1.0, 0.9, 0.8, 0.7]```**应用场景 (Use Cases)**:- 非线性可分的数据- 需要捕捉局部结构- 不知道数据的具体结构时的默认选择---### 3.3 核函数对比表 (Kernel Comparison Table)| Kernel类型 | 公式 | 计算复杂度 | 映射维度 | 论文使用 ||-----------|------|-----------|---------|---------|| **Linear** | $\mathbf{x}^T\mathbf{x}'$ | O(D) | D | ✅ 默认 || **RBF** | $\exp(-\|\mathbf{x}-\mathbf{x}'\|^2/2\sigma^2)$ | O(D) | ∞ | 可选 || **Polynomial** | $(\mathbf{x}^T\mathbf{x}'+c)^d$ | O(D) | $\binom{D+d}{d}$ | 很少 || **Cosine** | $\frac{\mathbf{x}^T\mathbf{x}'}{\|\mathbf{x}\|\|\mathbf{x}'\|}$ | O(D) | D | 很少 |

---<a id='section4'></a>## 4. Kernel在CKA中的应用 (Kernel in CKA)### 4.1 CKA的核心公式 (CKA Core Formula)**论文 Equation (28)**:$$m_{\text{CKA}}(R, R') = \frac{\text{HSIC}(K, L)}{\sqrt{\text{HSIC}(K, K) \cdot \text{HSIC}(L, L)}}$$其中 (where):- $K = \phi(R)\phi(R)^T$ - R的核矩阵- $L = \phi(R')\phi(R')^T$ - R'的核矩阵### 4.2 为什么CKA需要Kernel?**问题 (Problem)**:```神经网络1的表示 R1: 50样本 × 100维神经网络2的表示 R2: 50样本 × 80维维度不同！无法直接比较！```**解决方案 (Solution)**:```python# 使用Linear KernelK1 = R1 @ R1.T  # 50 × 50  ✓K2 = R2 @ R2.T  # 50 × 50  ✓# 现在可以用CKA比较了！CKA_score = cka(K1, K2)```### 4.3 Linear CKA vs RBF CKA**Linear CKA** (论文默认):```pythonK = R @ R.T  # 线性核L = R' @ R'.T```**RBF CKA** (可选):```pythonfrom scipy.spatial.distance import cdist# 计算距离矩阵D_R = cdist(R, R)D_R_prime = cdist(R', R')# RBF kernelK = np.exp(-D_R**2 / (2 * sigma**2))L = np.exp(-D_R_prime**2 / (2 * sigma**2))```### 4.4 论文中的结论**Kornblith et al. (2019)** 发现:> "Linear and RBF kernels give similar results for CKA"> > 线性核和RBF核给出相似的结果**因此**:- 论文主要使用 **Linear Kernel**- 更快、更简单、结果相似

---<a id='section5'></a>## 5. 代码实现 (Code Implementation)让我们从头开始实现各种Kernel函数！

In [ ]:
# 导入必要的库import numpy as npimport matplotlib.pyplot as pltfrom scipy.spatial.distance import cdist, pdistimport seaborn as sns# 设置随机种子以便复现np.random.seed(42)# 设置matplotlib显示中文plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']plt.rcParams['axes.unicode_minus'] = Falseprint("✓ Libraries imported successfully!")print(f"NumPy version: {np.__version__}")

### 5.1 核函数类实现 (Kernel Functions Class)

In [ ]:
class KernelFunctions:    """    常用核函数的实现    Implementation of common kernel functions    """        @staticmethod    def linear(X1, X2):        """        线性核: K(x, x') = x^T x'        Linear kernel: K(x, x') = x^T x'                参数 (Parameters):            X1: (N1 x D) 矩阵            X2: (N2 x D) 矩阵                返回 (Returns):            K: (N1 x N2) 核矩阵        """        return X1 @ X2.T        @staticmethod    def rbf(X1, X2, sigma=1.0):        """        RBF核/高斯核: K(x, x') = exp(-||x-x'||^2 / (2*sigma^2))        RBF/Gaussian kernel                参数 (Parameters):            X1: (N1 x D) 矩阵            X2: (N2 x D) 矩阵            sigma: 带宽参数 (bandwidth parameter)                返回 (Returns):            K: (N1 x N2) 核矩阵        """        distances_sq = cdist(X1, X2, metric='sqeuclidean')        return np.exp(-distances_sq / (2 * sigma**2))        @staticmethod    def polynomial(X1, X2, degree=2, coef0=1.0):        """        多项式核: K(x, x') = (x^T x' + coef0)^degree        Polynomial kernel                参数 (Parameters):            X1: (N1 x D) 矩阵            X2: (N2 x D) 矩阵            degree: 多项式度数            coef0: 常数项                返回 (Returns):            K: (N1 x N2) 核矩阵        """        return (X1 @ X2.T + coef0) ** degree        @staticmethod    def cosine(X1, X2):        """        余弦核: K(x, x') = (x^T x') / (||x|| ||x'||)        Cosine kernel (cosine similarity)                参数 (Parameters):            X1: (N1 x D) 矩阵            X2: (N2 x D) 矩阵                返回 (Returns):            K: (N1 x N2) 核矩阵        """        X1_norm = X1 / np.linalg.norm(X1, axis=1, keepdims=True)        X2_norm = X2 / np.linalg.norm(X2, axis=1, keepdims=True)        return X1_norm @ X2_norm.T# 创建实例kernel = KernelFunctions()print("✓ KernelFunctions class defined successfully!")

### 5.2 测试核函数 (Test Kernel Functions)

In [ ]:
# 创建测试数据n_samples = 5n_features = 3X1 = np.random.randn(n_samples, n_features)X2 = np.random.randn(4, n_features)print("="*60)print("测试核函数 (Testing Kernel Functions)")print("="*60)# Linear kernelK_linear = kernel.linear(X1, X2)print(f"\n1. Linear Kernel:")print(f"   输入形状: X1={X1.shape}, X2={X2.shape}")print(f"   输出形状: K={K_linear.shape}")print(f"   前3个值: {K_linear[0, :3]}")# RBF kernelK_rbf = kernel.rbf(X1, X2, sigma=1.0)print(f"\n2. RBF Kernel (σ=1.0):")print(f"   输入形状: X1={X1.shape}, X2={X2.shape}")print(f"   输出形状: K={K_rbf.shape}")print(f"   范围: [{K_rbf.min():.3f}, {K_rbf.max():.3f}]")print(f"   前3个值: {K_rbf[0, :3]}")# Polynomial kernelK_poly = kernel.polynomial(X1, X2, degree=2)print(f"\n3. Polynomial Kernel (d=2):")print(f"   输入形状: X1={X1.shape}, X2={X2.shape}")print(f"   输出形状: K={K_poly.shape}")print(f"   前3个值: {K_poly[0, :3]}")# Cosine kernelK_cosine = kernel.cosine(X1, X2)print(f"\n4. Cosine Kernel:")print(f"   输入形状: X1={X1.shape}, X2={X2.shape}")print(f"   输出形状: K={K_cosine.shape}")print(f"   范围: [{K_cosine.min():.3f}, {K_cosine.max():.3f}]")print(f"   前3个值: {K_cosine[0, :3]}")print("\n" + "="*60)print("✓ 所有核函数测试完成！")print("="*60)

---<a id='section6'></a>## 6. 可视化理解 (Visual Understanding)让我们通过可视化来直观理解不同的核函数！

### 6.1 核矩阵可视化 (Kernel Matrix Visualization)

In [ ]:
# 创建更大的数据集用于可视化np.random.seed(42)n_samples = 100n_features = 2X = np.random.randn(n_samples, n_features)# 计算不同的核矩阵K_linear = kernel.linear(X, X)K_rbf_small = kernel.rbf(X, X, sigma=0.5)K_rbf_large = kernel.rbf(X, X, sigma=2.0)# 创建可视化fig, axes = plt.subplots(2, 3, figsize=(15, 10))# 第一行: 原始数据ax = axes[0, 0]ax.scatter(X[:, 0], X[:, 1], alpha=0.6, s=50, c='steelblue')ax.set_title('原始数据 (Original Data)\n100个样本', fontsize=12, fontweight='bold')ax.set_xlabel('Feature 1')ax.set_ylabel('Feature 2')ax.grid(True, alpha=0.3)# 标注几个点for i in [0, 1, 50]:    ax.scatter(X[i, 0], X[i, 1], c='red', s=200, marker='*', zorder=5)    ax.annotate(f'{i}', (X[i, 0], X[i, 1]),                 xytext=(10, 10), textcoords='offset points',                fontsize=9, color='red', fontweight='bold')# Linear Kernelax = axes[0, 1]im = ax.imshow(K_linear, cmap='viridis', aspect='auto')ax.set_title('Linear Kernel\nK = X @ X^T', fontsize=12, fontweight='bold')ax.set_xlabel('Sample')ax.set_ylabel('Sample')plt.colorbar(im, ax=ax, label='Similarity')ax = axes[0, 2]ax.hist(K_linear.flatten(), bins=50, alpha=0.7, edgecolor='black', color='steelblue')ax.set_title('Linear Kernel\n相似度分布', fontsize=11, fontweight='bold')ax.set_xlabel('Similarity Value')ax.set_ylabel('Frequency')ax.axvline(np.mean(K_linear), color='red', linestyle='--',             label=f'Mean={np.mean(K_linear):.2f}', linewidth=2)ax.legend()ax.grid(True, alpha=0.3)# RBF Kernel (small sigma)ax = axes[1, 0]im = ax.imshow(K_rbf_small, cmap='viridis', aspect='auto', vmin=0, vmax=1)ax.set_title('RBF Kernel (σ=0.5)\n小带宽 - 局部相似', fontsize=12, fontweight='bold')ax.set_xlabel('Sample')ax.set_ylabel('Sample')plt.colorbar(im, ax=ax, label='Similarity [0,1]')# RBF Kernel (large sigma)ax = axes[1, 1]im = ax.imshow(K_rbf_large, cmap='viridis', aspect='auto', vmin=0, vmax=1)ax.set_title('RBF Kernel (σ=2.0)\n大带宽 - 全局相似', fontsize=12, fontweight='bold')ax.set_xlabel('Sample')ax.set_ylabel('Sample')plt.colorbar(im, ax=ax, label='Similarity [0,1]')# 距离 vs 相似度ax = axes[1, 2]sample_idx = 0distances = np.linalg.norm(X - X[sample_idx], axis=1)ax.scatter(distances, K_linear[sample_idx, :], alpha=0.5, label='Linear', s=30)ax.scatter(distances, K_rbf_small[sample_idx, :], alpha=0.5, label='RBF (σ=0.5)', s=30)ax.scatter(distances, K_rbf_large[sample_idx, :], alpha=0.5, label='RBF (σ=2.0)', s=30)ax.set_title(f'距离 vs 相似度\n(相对于样本{sample_idx})', fontsize=11, fontweight='bold')ax.set_xlabel('Euclidean Distance')ax.set_ylabel('Kernel Similarity')ax.legend()ax.grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n核矩阵统计 (Kernel Statistics):")print(f"\nLinear Kernel:")print(f"  范围: [{K_linear.min():.2f}, {K_linear.max():.2f}]")print(f"  均值: {K_linear.mean():.2f}")print(f"  标准差: {K_linear.std():.2f}")print(f"\nRBF (σ=0.5):")print(f"  范围: [{K_rbf_small.min():.2f}, {K_rbf_small.max():.2f}]")print(f"  均值: {K_rbf_small.mean():.2f}")print(f"  标准差: {K_rbf_small.std():.2f}")print(f"\nRBF (σ=2.0):")print(f"  范围: [{K_rbf_large.min():.2f}, {K_rbf_large.max():.2f}]")print(f"  均值: {K_rbf_large.mean():.2f}")print(f"  标准差: {K_rbf_large.std():.2f}")

### 6.2 Kernel在CKA中的作用 (Kernel's Role in CKA)

In [ ]:
# 模拟两个神经网络的表示np.random.seed(123)N = 50  # 样本数D1 = 100  # 第一个表示的维度D2 = 80   # 第二个表示的维度# 创建两个有一定相关性的表示R1 = np.random.randn(N, D1)R2 = np.random.randn(N, D2) + 0.3 * np.random.randn(N, D2)# 计算核矩阵K1 = R1 @ R1.TK2 = R2 @ R2.T# HSIC函数def hsic(K, L):    """Hilbert-Schmidt Independence Criterion"""    N = K.shape[0]    H = np.eye(N) - np.ones((N, N)) / N    K_centered = H @ K @ H    L_centered = H @ L @ H    return np.trace(K_centered @ L_centered) / ((N - 1) ** 2)# 计算CKAdef cka(K1, K2):    """Centered Kernel Alignment"""    hsic_k1_k2 = hsic(K1, K2)    hsic_k1_k1 = hsic(K1, K1)    hsic_k2_k2 = hsic(K2, K2)    return hsic_k1_k2 / np.sqrt(hsic_k1_k1 * hsic_k2_k2)cka_score = cka(K1, K2)# 可视化fig, axes = plt.subplots(2, 3, figsize=(15, 10))# 第一行: 原始表示axes[0, 0].imshow(R1[:20, :20], cmap='RdBu_r', aspect='auto')axes[0, 0].set_title(f'神经网络1的表示 R1\n形状: {R1.shape}', fontsize=11, fontweight='bold')axes[0, 0].set_xlabel('Feature Dimension')axes[0, 0].set_ylabel('Sample')axes[0, 1].imshow(R2[:20, :20], cmap='RdBu_r', aspect='auto')axes[0, 1].set_title(f'神经网络2的表示 R2\n形状: {R2.shape}', fontsize=11, fontweight='bold')axes[0, 1].set_xlabel('Feature Dimension')axes[0, 1].set_ylabel('Sample')axes[0, 2].text(0.5, 0.5,     f'问题:\n如何比较R1和R2？\n\n' +    f'R1: {R1.shape[0]}样本 × {R1.shape[1]}维\n' +    f'R2: {R2.shape[0]}样本 × {R2.shape[1]}维\n\n' +    f'维度不同！\n无法直接比较！',    ha='center', va='center', fontsize=13,    transform=axes[0, 2].transAxes,    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3))axes[0, 2].axis('off')# 第二行: 核矩阵im1 = axes[1, 0].imshow(K1, cmap='viridis', aspect='auto')axes[1, 0].set_title(f'核矩阵 K1 = R1 @ R1^T\n形状: {K1.shape}', fontsize=11, fontweight='bold')axes[1, 0].set_xlabel('Sample')axes[1, 0].set_ylabel('Sample')plt.colorbar(im1, ax=axes[1, 0], label='Similarity')im2 = axes[1, 1].imshow(K2, cmap='viridis', aspect='auto')axes[1, 1].set_title(f'核矩阵 K2 = R2 @ R2^T\n形状: {K2.shape}', fontsize=11, fontweight='bold')axes[1, 1].set_xlabel('Sample')axes[1, 1].set_ylabel('Sample')plt.colorbar(im2, ax=axes[1, 1], label='Similarity')axes[1, 2].text(0.5, 0.5,     f'解决方案: Kernel!\n\n' +    f'K1 = R1 @ R1^T → {K1.shape}\n' +    f'K2 = R2 @ R2^T → {K2.shape}\n\n' +    f'现在维度相同！\n可以用CKA比较！\n\n' +    f'CKA(K1, K2) = {cka_score:.3f}\n\n' +    f'✓ 捕捉了样本间关系\n' +    f'✓ 无需匹配特征维度\n' +    f'✓ 这就是Kernel的威力！',    ha='center', va='center', fontsize=11,    transform=axes[1, 2].transAxes,    bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))axes[1, 2].axis('off')plt.tight_layout()plt.show()print(f"\n✓ CKA相似度: {cka_score:.4f}")print(f"  (范围 [0, 1]: 1 = 完全相同, 0 = 完全不同)")

---<a id='section7'></a>## 7. 实际应用案例 (Practical Application)### 用Kernel比较基因表达数据 (Using Kernels for Gene Expression Data)假设你有两个实验条件下的基因表达数据：- **对照组 (Control)**: 正常条件- **处理组 (Treated)**: 重金属暴露我们想知道：重金属是否改变了基因表达模式？

In [ ]:
# 模拟基因表达数据# Simulate gene expression datanp.random.seed(42)n_samples_control = 100  # 对照组样本数n_samples_treated = 100  # 处理组样本数n_genes = 1000          # 基因数量# 对照组: 正常表达expression_control = np.random.randn(n_samples_control, n_genes)# 处理组: 部分基因表达改变expression_treated = np.random.randn(n_samples_treated, n_genes)# 假设前100个基因受重金属影响，表达显著上调expression_treated[:, :100] += 2.0  # 上调# 假设100-200个基因表达下调expression_treated[:, 100:200] -= 1.5  # 下调print("="*60)print("基因表达数据模拟 (Gene Expression Data Simulation)")print("="*60)print(f"\n对照组 (Control): {expression_control.shape}")print(f"  - {n_samples_control} 个样本")print(f"  - {n_genes} 个基因")print(f"\n处理组 (Treated): {expression_treated.shape}")print(f"  - {n_samples_treated} 个样本")  print(f"  - {n_genes} 个基因")print(f"  - 前100个基因: 上调 (+2.0)")print(f"  - 100-200基因: 下调 (-1.5)")print(f"  - 其余基因: 无变化")

In [ ]:
# 计算核矩阵K_control = kernel.linear(expression_control, expression_control)K_treated = kernel.linear(expression_treated, expression_treated)# 也可以尝试RBF kernelK_control_rbf = kernel.rbf(expression_control, expression_control, sigma=10.0)K_treated_rbf = kernel.rbf(expression_treated, expression_treated, sigma=10.0)# 计算CKA - 比较两个条件cka_linear = cka(K_control, K_treated)cka_rbf = cka(K_control_rbf, K_treated_rbf)print("\n" + "="*60)print("CKA分析结果 (CKA Analysis Results)")print("="*60)print(f"\nLinear CKA: {cka_linear:.4f}")print(f"RBF CKA:    {cka_rbf:.4f}")print(f"\n解释 (Interpretation):")if cka_linear > 0.7:    print(f"  ✓ 高相似度 (>0.7): 基因表达模式基本相同")elif cka_linear > 0.4:    print(f"  ⚠ 中等相似度 (0.4-0.7): 有一定差异但总体模式相似")else:    print(f"  ✗ 低相似度 (<0.4): 基因表达模式显著不同")print(f"\n在这个例子中:")print(f"  - CKA = {cka_linear:.4f} 说明处理组改变了基因表达")print(f"  - 但整体样本间的关系结构还保留了一定相似性")

In [ ]:
# 可视化比较fig, axes = plt.subplots(2, 3, figsize=(15, 10))# 第一行: 基因表达热图axes[0, 0].imshow(expression_control[:50, :50], cmap='RdBu_r', aspect='auto', vmin=-3, vmax=3)axes[0, 0].set_title('对照组表达 (Control)\n(50样本 × 50基因)', fontsize=11, fontweight='bold')axes[0, 0].set_xlabel('Gene')axes[0, 0].set_ylabel('Sample')axes[0, 1].imshow(expression_treated[:50, :50], cmap='RdBu_r', aspect='auto', vmin=-3, vmax=3)axes[0, 1].set_title('处理组表达 (Treated)\n(50样本 × 50基因)', fontsize=11, fontweight='bold')axes[0, 1].set_xlabel('Gene')axes[0, 1].set_ylabel('Sample')# 差异difference = np.mean(expression_treated, axis=0) - np.mean(expression_control, axis=0)axes[0, 2].bar(range(100), difference[:100], color='steelblue', alpha=0.7)axes[0, 2].axhline(0, color='red', linestyle='--', linewidth=2)axes[0, 2].set_title('基因表达差异\n(前100个基因)', fontsize=11, fontweight='bold')axes[0, 2].set_xlabel('Gene Index')axes[0, 2].set_ylabel('Expression Difference\n(Treated - Control)')axes[0, 2].grid(True, alpha=0.3, axis='y')# 第二行: 核矩阵im1 = axes[1, 0].imshow(K_control, cmap='viridis', aspect='auto')axes[1, 0].set_title(f'对照组核矩阵\nLinear Kernel', fontsize=11, fontweight='bold')axes[1, 0].set_xlabel('Sample')axes[1, 0].set_ylabel('Sample')plt.colorbar(im1, ax=axes[1, 0])im2 = axes[1, 1].imshow(K_treated, cmap='viridis', aspect='auto')axes[1, 1].set_title(f'处理组核矩阵\nLinear Kernel', fontsize=11, fontweight='bold')axes[1, 1].set_xlabel('Sample')axes[1, 1].set_ylabel('Sample')plt.colorbar(im2, ax=axes[1, 1])# CKA结果总结axes[1, 2].text(0.5, 0.5,    f'CKA分析结果\n\n' +    f'Linear CKA: {cka_linear:.3f}\n' +    f'RBF CKA: {cka_rbf:.3f}\n\n' +    f'{'='*30}\n\n' +    f'生物学意义:\n' +    f'• 处理组改变了基因表达\n' +    f'• 但样本间关系保留\n' +    f'• 可能是部分基因受影响\n' +    f'• 需要进一步分析差异基因',    ha='center', va='center', fontsize=11,    transform=axes[1, 2].transAxes,    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3),    family='monospace')axes[1, 2].axis('off')plt.tight_layout()plt.show()print("\n✓ 可视化完成！")

---<a id='section8'></a>## 8. 总结 (Summary)### 🎯 核心要点 (Key Takeaways)#### 1. Kernel是什么？- **Kernel = 相似度函数**- 衡量两个数据点有多相似- 公式: $K(\mathbf{x}, \mathbf{x}') \rightarrow$ 相似度分数#### 2. 为什么需要Kernel？- **Kernel Trick**: 不显式计算高维映射- 直接在原始空间计算内积- 效率高：$O(D)$ vs $O(D_{high})$ 甚至 $O(\infty)$#### 3. 论文中的应用```python# CKA使用Linear Kernel (默认)K1 = R1 @ R1.T  # 神经网络1的核矩阵K2 = R2 @ R2.T  # 神经网络2的核矩阵# 计算CKA相似度CKA_score = cka(K1, K2)```#### 4. 常用Kernel对比| Kernel | 何时使用 | 优点 | 缺点 ||--------|---------|------|------|| **Linear** | 数据线性可分 | 快速、简单、论文默认 | 无法捕捉非线性 || **RBF** | 非线性数据 | 强大、通用 | 需要调参σ || **Polynomial** | 需要多项式特征 | 可控制度数 | 高度数易过拟合 || **Cosine** | 文本、向量 | 归一化相似度 | 忽略幅度信息 |---### 📊 在你的研究中应用 (Application in Your Research)#### RNA-seq / 基因表达数据```python# 比较不同条件的基因表达模式K_condition1 = gene_expr_1 @ gene_expr_1.TK_condition2 = gene_expr_2 @ gene_expr_2.Tsimilarity = cka(K_condition1, K_condition2)# → 告诉你两个条件下基因表达模式的相似度```#### 重金属毒理学研究```python# 比较不同重金属的毒性机制K_cadmium = response_Cd @ response_Cd.TK_mercury = response_Hg @ response_Hg.Tsimilarity = cka(K_cadmium, K_mercury)# → 相似度高 = 毒性机制相似# → 相似度低 = 毒性机制不同```---### 💡 实用建议 (Practical Recommendations)1. **默认选择**: 先用 **Linear Kernel**   - 简单、快速、结果好   - 论文的默认选择2. **非线性数据**: 尝试 **RBF Kernel**   - 参数选择: median heuristic   - `sigma = np.median(pdist(X))`3. **多试几个**: Cross-validation   - 比较不同kernel的结果   - 选择最稳定的4. **生物解释**:    - 高CKA → 相似的生物学过程   - 低CKA → 不同的分子机制   - 中等CKA → 部分共享通路---### 📚 延伸阅读 (Further Reading)1. **Kornblith et al. (2019)**: "Similarity of Neural Network Representations Revisited"   - CKA的原始论文   - 详细的理论分析2. **Klabunde et al. (2025)**: "Similarity of Neural Network Models: A Survey"   - 53+种相似度度量   - 全面的综述3. **Schölkopf & Smola**: "Learning with Kernels"   - Kernel方法的经典教材   - 深入的数学理论---### ✅ 下一步 (Next Steps)1. 用你自己的数据试试这些kernel2. 比较Linear vs RBF的结果3. 尝试用CKA分析不同实验条件4. 可视化核矩阵理解数据结构---## 🎓 恭喜！你已经掌握了Kernel的核心概念！如有任何问题，随时问我！😊

In [ ]:
# 这个notebook的完整代码已经准备好！# 你可以:# 1. 修改参数试试不同的kernel# 2. 用你自己的数据替换示例数据# 3. 添加更多的可视化# 4. 尝试其他类型的kernelprint("="*60)print("✓ Kernel Tutorial 完成！")print("="*60)print("\n感谢学习！如有问题随时提问！😊")